In [1]:
import os
import datetime
from datetime import timedelta
import time
import pandas as pd
import numpy as np
from pandas import DataFrame
from datetime import datetime

time_run = time.time()

In [2]:
xls = pd.ExcelFile("../Input file/Costing Examples_NEW.xlsx")
time_df = pd.read_excel(xls, 'Check')
holiday_df = xls.parse("Holiday")
holiday_list = [("" if m>9 else "0") + str(m) + "/" + ("" if d>9 else "0") + str(d) for d, m in zip(holiday_df["Day"], holiday_df["Month"])]

base_rate = 20
holiday_rate = 2.5

rate1 = np.array([[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1.5,1.5,1.5,1.5,2],[2,2,2,2,2]])
rate2 = np.array([[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1.5,1.5,1.5,1.5,2],[2,2,2,2,2]])
rate3 = np.array([[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.7,1.7,1.7,1.7,2.2],[2.2,2.2,2.2,2.2,2.2]])   
rate_data = {"Full Time": rate1, "Part Time": rate2, "Casual": rate3}

In [3]:
def date_to_str(date):
    date = str(date).split("-")
    return date[1] + "/" + date[2] 

def day_val(day):
    val = {"Monday":0,"Tuesday":1,"Wednesday":2,'Thursday':3,"Friday":4,"Saturday":5,"Sunday":6}
    return val[day]

def cal_base_rate(_type, day, date, start_time, end_time, rate_data):
    
    shift = 2;      
    ord_rate = 0
    
    if date_to_str(date) in holiday_list:
        ord_rate = holiday_rate
    else:        
        if _type == "Full Time":
            if start_time < 5 or end_time > 1:
                shift = 2
            elif end_time > 19 or end_time < 1:
                shift = 1
            elif start_time >= 5 and end_time < 20:
                shift = 0
            
        elif _type == "Part Time":
            
            if start_time < 5 or end_time > 1:
                shift = 2
            elif end_time > 19 or end_time < 1:
                shift = 1
            elif start_time >= 5 and end_time < 20:
                shift = 0

        else:
            
            if start_time < 5 or end_time > 1:
                shift = 2
            elif end_time > 19 or end_time < 1:
                shift = 1
            elif start_time >= 5 and end_time < 20:
                shift = 0
            
        ord_rate = rate_data[_type][day_val(day)][shift]
            
    return ord_rate

In [4]:
   
data= []
list_pay = []
for row in time_df.iterrows():    
    _id = row[1]["Contact Name"] 
    
    start_period = row[1]['Process period start date'].date()
    end_period = row[1]['Process period end date'].date()
    
    start = row[1]["Start date"]
    end   = row[1]["End date"]   
    start_time = start.hour + start.minute/60
    end_time = end.hour + end.minute/60
        
    _type = row[1]["Employment Type"]
    
    start_date = start.date() 
    end_date   = end.date()
    
    delta_day = int((start_date- end_date).days)
    
    start_day  = start.strftime('%A')
    end_day    = end.strftime('%A')  
        # ý là trc cái or cách kia kai là 1 sau là 1 cơ mà đều có start dte khác end date ý nên gộp lại start khác date and (()or())
    if ((start_date != end_date) and ( ((end_time >=1) and (start_day in ["Friday","Saturday","Sunday"])) or ( date_to_str(start_date) in holiday_list or date_to_str(end_date) in holiday_list ) )):
        
        rate1 = cal_base_rate(_type, start_day, start_date, start_time, 24, rate_data)
        rate2 = cal_base_rate(_type, end_day, end_date, 0, end_time, rate_data)
        delta_time1 = 24 - start_time
        delta_time2 = end_time      # à cái rate chỉ dựa vaod start thoi đúng k ta :) sửa lại hàm rate hả rate dựa vô start date hoi a
        start_date1 = start_date    # là sao
        end_date1= end_date
        start_date2 = end_date
        end_date2= end_date
        
        start1= start
        end2=end
        end1= end.replace(hour=0,minute=0)
        start2=end1
        
        start_day1=start1.strftime('%A')
        end_day1=end1.strftime('%A')
        start_day2=start2.strftime('%A')
        end_day2=end2.strftime('%A')
        
        
        
        data.append((_id,start_period,end_period,start_date1,end_date1,start1,end1,delta_time1, rate1))
        data.append((_id,start_period,end_period,start_date2,end_date2, start2, end2,delta_time2, rate2))  
    
    else: 
        delta_time = (end-start).total_seconds() / 3600.0
        rate = cal_base_rate(_type, start_day, start_date, start_time, end_time, rate_data)
        
        
        data.append((_id,start_period,end_period,start_date,end_date,start,end,delta_time, rate))
            
            
file = pd.DataFrame(data,columns=['ID',"Start Period",'End Period','Start Date','End Date','Start','End','Hour','Rate'])        


    

In [44]:
file = file.sort_values(by = ["ID","Start Period"])
len(file["Start Period"].unique())

# for row in file.iterrows():
#     ""

2

In [42]:
total_hour_per_day= file.groupby(['ID','Start Period','End Period',"Start Date"]).ngroup()
total_hour_per_day
# len(total_hour_per_day["Start Period"])

0     1
1     1
2     2
6     0
8     3
3     5
4     6
5     5
7     4
9     7
10    8
dtype: int64

In [7]:
total_hour_per_day.loc[total_hour_per_day['Hour'] < 3]

,,,,Hour
ID,Start Period,End Period,Start Date,
Aaron Edwards,2021-03-22,2021-03-28,2021-03-27,2.0
Quynh,2021-04-22,2021-04-28,2021-04-25,0.5


In [8]:
for name in total_hour_per_day:
    if name<=3:
        name = 3
        print(name)
    else: 
        name = name
        print(name)

TypeError: '<=' not supported between instances of 'str' and 'int'